In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
import matplotlib.pyplot as plt # data visualization
import seaborn as sns #data visualization
import os

from sklearn.metrics import classification_report, confusion_matrix
from PIL import Image

import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.models import Model,load_model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions

In [ ]:
# cwd = os.getcwd()  # Get the current working directory (cwd)
# print(cwd)
# files = os.listdir(cwd) 
# print(files) 
# os.chdir("/home/nikosili/projects/datatalks_zoomcamp/CNN/chest_xray")
# print(cwd)

In [ ]:
dir_ = os.getcwd()
data = os.path.join(dir_, 'data')
flowers = os.listdir(data) 
for i in flowers:
    print(i,'=',len(os.listdir(os.path.join(dir_, 'data', i))))

In [ ]:
input_tensor  = Input(shape=(224,224,3))
img_size      = (224, 224)
batch_size    = 32  
num_epochs    = 50
learning_rate = 0.0001

img_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=50,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip = True ,
                    vertical_flip = True ,
                    fill_mode='nearest',
                    validation_split=0.2)


train_generator = img_datagen.flow_from_directory(
        data,  # training data directory
        target_size=img_size,  
        batch_size=batch_size,
        shuffle=True,
        # because it is a classification problem more 2 classes then use class_mode = 'categorical'
        class_mode='categorical',
        subset='training') # set as training data
 
validation_generator = img_datagen.flow_from_directory(
        data, # validation data directory
        target_size=img_size, 
        batch_size=batch_size,
        shuffle=False,
        # because it is a classification problem more 2 classes then use class_mode = 'categorical'
        class_mode='categorical',
        subset='validation') # set as validation data

In [ ]:
base_model = Xception(weights='imagenet', pooling='avg', include_top=False,input_shape=(224, 224, 3))

In [ ]:
base_model.trainable = True

model = models.Sequential()
model.add(base_model)
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(7, activation='softmax')) #softmax for multi-class classification


model.summary()

In [ ]:
# Checkpoint

model_file_path='./model.hdf5'

checkpoints = ModelCheckpoint(model_file_path,monitor='val_accuracy',
                                  mode='max',save_best_only=True)

In [ ]:
# Model Compile

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(learning_rate=learning_rate),
              metrics=['accuracy'])

In [ ]:
# Fit Model
print(len(train_generator))
print(len(validation_generator))

steps_per_epoch=train_generator.n/batch_size
validation_steps=validation_generator.n/batch_size

result = model.fit(
      train_generator,
      steps_per_epoch=steps_per_epoch,
      epochs=num_epochs, 
      validation_data=validation_generator,
      validation_steps=validation_steps,
      verbose=1, 
      callbacks=[checkpoints]
      )

In [ ]:
data